In [1]:
import os
import re
from pathlib import Path
import pickle
from IPython.display import clear_output
import neurox.data.loader as data_loader
import neurox.interpretation.utils as utils
import neurox.interpretation.ablation as ablation
import neurox.interpretation.linear_probe as linear_probe
import neurox.data.extraction.transformers_extractor as transformers_extractor

In [2]:
path_project = Path(os.getcwd()).parents[0]
path_project = str(path_project)

In [3]:
data_path = path_project+'/large_data_en_gum'

In [4]:
directories = [] #тут делаем обход, чтобы получить имена каталогов
for root,dirs,files in os.walk(data_path):
    for directory in dirs:
        directories.append(os.path.join(root, directory))

In [5]:
all_paths = []
for directory in directories: #тут делаем обход каталагов, чтобы получить имена файлов
    paths = []
    for (root,dirs,files) in os.walk(directory, topdown=True):
        for file in files:
            paths.append(os.path.join(root, file))
        all_paths.append(sorted(paths))

In [6]:
def load_sentence_data(source_path, labels_path, activations): 
    
    #тут немного переписали функцию потому что в библиотеке ошибка!!!

    tokens = {"source": [], "target": []}

    with open(source_path) as source_fp:
        for line_idx, line in enumerate(source_fp):
            line_tokens = line.strip().split() #вот тут переписано
            tokens["source"].append(line_tokens) #и тут

    with open(labels_path) as labels_fp:
        for line in labels_fp:
            line_tokens = line.strip().split()
            tokens["target"].append(line_tokens)

    assert len(tokens["source"]) == len(tokens["target"]), (
        "Number of lines do not match (source: %d, target: %d)!"
        % (len(tokens["source"]), len(tokens["target"]))
    )

    assert len(activations) == len(tokens["source"]), (
        "Number of lines do not match (activations: %d, source: %d)!"
        % (len(activations), len(tokens["source"]))
    )

    
    for idx, activation in enumerate(activations):
        assert activation.shape[0] == len(tokens["source"][idx])

    return tokens

In [7]:
class Experiment:
    
    
    def __init__(self, path_trdata, path_trlabel, path_tedata, path_telabel):
        
        #некрасиво инициализирована куча переменных для функций
        
        self.path_trdata = path_trdata
        self.path_trlabel = path_trlabel
        self.path_tedata = path_tedata
        self.path_telabel = path_telabel
        self.category = re.search(r'(?<=_)[a-zA-Z]+(?=.txt)', path_trdata)[0]
        self.dataset = re.search(r'(?<=_)[a-zA-Z]+_[a-zA-Z]+(?=\/)', path_trdata)[0]
        
        self.path = path_project+f'/large_data_{self.dataset}/data_{self.category}'
        
        self.activations_tr, self.num_layers = data_loader.load_activations(self.path+'/activations_train.json', 768)
        self.activations_te, self.num_layers = data_loader.load_activations(self.path+'/activations_te.json', 768)
        
        self.tokens_tr = load_sentence_data(self.path_trdata, self.path_trlabel, self.activations_tr)
        self.tokens_te = load_sentence_data(self.path_tedata, self.path_telabel, self.activations_te)
        
        self.X_tr, self.y_tr, self.mapping_tr = utils.create_tensors(self.tokens_tr, self.activations_tr, 'Nom')
        self.label2idx_tr, self.idx2label_tr, self.src2idx_tr, self.idx2src_tr = self.mapping_tr
        
        self.X_te, self.y_te, self.mapping_te = utils.create_tensors(self.tokens_te, self.activations_te, 'Nom')
        self.label2idx_te, self.idx2label_te, self.src2idx_te, self.idx2src_te = self.mapping_te
        
        
    def run_classification(self):#just пробинг
        probe = linear_probe.train_logistic_regression_probe(self.X_tr, self.y_tr, lambda_l1=0.001, lambda_l2=0.001)
        scores_tr = linear_probe.evaluate_probe(probe, self.X_tr, self.y_tr, idx_to_class=self.idx2label_tr)
        scores_te = linear_probe.evaluate_probe(probe, self.X_te, self.y_te, idx_to_class=self.idx2label_te)
        return probe, scores_tr, scores_te
    
    
    def lairwise(self, n): #пробинг по слоям! можно выбрать
        layer_0_X_tr = ablation.filter_activations_by_layers(self.X_tr, [n], 13)
        probe_layer_0 = linear_probe.train_logistic_regression_probe(layer_0_X_tr, self.y_tr, lambda_l1=0.001, lambda_l2=0.001)
        scores_tr = linear_probe.evaluate_probe(probe_layer_0, layer_0_X_tr, self.y_tr, idx_to_class=self.idx2label_tr)
        
        layer_0_X_te = ablation.filter_activations_by_layers(self.X_te, [n], 13)
        scores_te = linear_probe.evaluate_probe(probe_layer_0, layer_0_X_te, self.y_te, idx_to_class=self.idx2label_te)
        return scores_tr, scores_te
    
    
    def nranking(self, n=100, k = 'ordering'): #тут топ нейроны
        probe, scores_tr, scores_te = self.run_classification()
        ordering, cutoffs = linear_probe.get_neuron_ordering(probe, self.label2idx_tr)
        if k == 'ordering':
            return ordering, scores_tr, scores_te
        elif k == 'train':
            X_tr_selected = ablation.filter_activations_keep_neurons(self.X_tr, ordering[:n])
            probe_selected = linear_probe.train_logistic_regression_probe(X_tr_selected, self.y_tr, lambda_l1=0.001, lambda_l2=0.001)
            scores_tr = linear_probe.evaluate_probe(probe_selected, X_tr_selected, self.y_tr, idx_to_class=self.idx2label_tr)
            X_te_selected = ablation.filter_activations_keep_neurons(self.X_te, ordering[:n])
            scores_te = linear_probe.evaluate_probe(probe_selected, X_te_selected, self.y_te, idx_to_class=self.idx2label_te)
            return ordering, scores_tr, scores_te

In [8]:
# datatest[2], labeltest[4]
# datatrain[3], labeltrain[5]
ordered_neurons = {}
scores = {}
for path in all_paths:
    cat = Experiment(path[3], path[5], path[2], path[4])    
    cat_name = re.search(r'(?<=_)[a-zA-Z]+(?=.txt)', path[3])[0]
    neurons, scores_tr, scores_te = cat.nranking()
    ordered_neurons[cat_name] = neurons
    scores[cat_name] = [scores_tr, scores_te] 

Loading json activations from /home/senya/Документы/project/large_data_en_gum/data_Number/activations_train.json...
2553 13.0
Loading json activations from /home/senya/Документы/project/large_data_en_gum/data_Number/activations_te.json...
882 13.0
Number of tokens:  2553
length of source dictionary:  8554
length of target dictionary:  2
2553
Total instances: 2553
['Significant', 'Prior', 'Some', 'Previous', 'Leave', 'Beyond', 'On', 'Sunday', 'Besides', 'Arrogant', 'Hackers', 'Sensations', 'Brampton', 'Italian', 'Achieving', 'Makes', 'Street', '"We', 'Europe', '"Do']
Number of samples:  2553
Stats: Labels with their frequencies in the final set
Sing 1300
Plur 1253
Number of tokens:  882
length of source dictionary:  4076
length of target dictionary:  2
882
Total instances: 882
['Southerners', 'elderly', 'Even', 'Some', 'Leave', 'Space', 'Second', 'Use', 'Later', 'On', 'Village', 'Besides', 'Mandatory', 'More', 'Uh', 'Sociologists', 'These', 'Closing', 'North', 'My']
Number of samples:  

epoch [1/10]: 0it [00:00, ?it/s]

Epoch: [1/10], Loss: 0.0212


epoch [2/10]: 0it [00:00, ?it/s]

Epoch: [2/10], Loss: 0.0152


...

Evaluating: 0it [00:00, ?it/s]

Score (accuracy) of the probe: 0.94


Evaluating: 0it [00:00, ?it/s]

Score (accuracy) of the probe: 0.77


  0%|          | 0/101 [00:00<?, ?it/s]

In [9]:
with open('scores_gum.pkl', 'wb') as f:
    pickle.dump(scores, f)
with open('neurons_gum.pkl', 'wb') as f:
    pickle.dump(ordered_neurons, f)